In [1]:
import pandas as pd
import numpy as np
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine

In [2]:
# Postgres username, password, and database name
POSTGRES_ADDRESS = '127.0.0.1' ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'postgres' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = '*****' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD 
POSTGRES_DBNAME = 'moodle' ## CHANGE THIS TO YOUR DATABASE NAME
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,
                                                                                        password=POSTGRES_PASSWORD,
                                                                                        ipaddress=POSTGRES_ADDRESS,
                                                                                        port=POSTGRES_PORT,
                                                                                        dbname=POSTGRES_DBNAME))
# Create the connection
engine = create_engine(postgres_str)


In [3]:
data = pd.read_sql_query("""
                        SELECT *, 
                        TO_CHAR('epoch'::timestamp + (timecreated ||' seconds')::interval, 'YYYY-MM-DD')
                        FROM mdl_logstore_standard_log;
                        """, engine)

In [4]:
data.head()

,id,eventname,component,action,target,objecttable,objectid,crud,edulevel,contextid,...,contextinstanceid,userid,courseid,relateduserid,anonymous,other,timecreated,origin,ip,realuserid
0,1,\core\event\course_viewed,core,viewed,course,None,NaN,r,2,2,...,1,0,1,NaN,0,N;,1548311114,web,105.233.48.139,NaN
1,2,\core\event\user_loggedin,core,loggedin,user,user,2.0,r,0,1,...,0,2,0,NaN,0,"a:1:{s:8:""username"";s:5:""admin"";}",1548311148,web,105.233.48.139,NaN
2,3,\core\event\dashboard_viewed,core,viewed,dashboard,None,NaN,r,0,5,...,2,2,0,2.0,0,N;,1548311179,web,105.233.48.139,NaN
3,4,\core\event\course_viewed,core,viewed,course,None,NaN,r,2,2,...,1,0,1,NaN,0,N;,1548313994,web,105.233.48.139,NaN
4,5,\core\event\course_viewed,core,viewed,course,None,NaN,r,2,2,...,1,0,1,NaN,0,N;,1548319150,web,41.13.254.147,NaN


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417554 entries, 0 to 417553
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 417554 non-null  int64  
 1   eventname          417554 non-null  object 
 2   component          417554 non-null  object 
 3   action             417554 non-null  object 
 4   target             417554 non-null  object 
 5   objecttable        362981 non-null  object 
 6   objectid           362981 non-null  float64
 7   crud               417554 non-null  object 
 8   edulevel           417554 non-null  int64  
 9   contextid          417554 non-null  int64  
 10  contextlevel       417554 non-null  int64  
 11  contextinstanceid  417554 non-null  int64  
 12  userid             417554 non-null  int64  
 13  courseid           417554 non-null  int64  
 14  relateduserid      325198 non-null  float64
 15  anonymous          417554 non-null  int64  
 16  ot

In [ ]:
class detl:
    def __init__(self, data):
        self.data = data
        
    def dedication_time(self):
        data = self.data
        data_time = data.groupby('id').agg({'hour':'sum'}).reset_index()
        return data_time
    
    def login_activity_count(self):
        data = self.data
        data['logged_in'] = data['action']. apply(lambda x: 1 if (x == 'loggedin') else 0)
        data['activity'] = data['action']. apply(lambda x: 1 if (x != 'loggedin') else 0)
        login = data.groupby('userid').agg({"logged_in":'sum',"activity":"sum"}).reset_index()
        return login
    
    def group_students(self, login, feature):
        login = self.data
        top_1 = login.groupby(feature).quantile(0.01)
        top_5 = login.groupby(feature).quantile(0.05)
        top_10 = login.groupby(feature).quantil(0.10)
        top_25 = login.groupby(feature).quantile(0.25)
        return top_1, top_5, top_10, top_25
    

        